In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
import pandas as pd

# Establish Snowflake session
session = st.connection('snowflake').session()

# Function to fetch data from Snowflake
def fetch_data():
    query = """
    SELECT FULL_NAME, PHONE_NUMBER
    FROM CANDIDATE_PROFILES_EDIT
    """
    df = session.sql(query).to_pandas()
    return df

# Function to update data in Snowflake (to CANDIDATE_PROFILES_EDIT table)
def update_data_in_snowflake(old_name, new_name, new_phone_number):
    query = f"""
    UPDATE CANDIDATE_PROFILES_EDIT
    SET FULL_NAME = '{new_name}', PHONE_NUMBER = '{new_phone_number}'
    WHERE FULL_NAME = '{old_name}'
    """
    session.sql(query).collect()  # Execute the update query

# Function to reload data from the database
def reload_data():
    return fetch_data()

# Fetch the initial data
if 'data' not in st.session_state:
    st.session_state.data = fetch_data()  # Store the data in session state if it doesn't exist

# Editable table UI
st.title("Edit Candidate Data")

# Display the data in the app
st.write("Current Candidate Data:")
st.dataframe(st.session_state.data)

# Create editable table
edited_data = st.session_state.data.copy()  # Make a copy of the data to be edited

# Loop through the DataFrame and allow row edits
for index, row in edited_data.iterrows():
    with st.expander(f"{index + 0}: {row['FULL_NAME']}"):
        # Editable fields for each row
        edited_data.at[index, 'FULL_NAME'] = st.text_input(f"Full Name", value=row['FULL_NAME'], key=f"name_{index}")
        edited_data.at[index, 'PHONE_NUMBER'] = st.text_input(f"Phone Number", value=row['PHONE_NUMBER'], key=f"phone_{index}")

# Button to save changes
if st.button("Save Changes"):
    # Update the database with the new values
    try:
        for index, row in edited_data.iterrows():
            if row['FULL_NAME'] != st.session_state.data.at[index, 'FULL_NAME'] or row['PHONE_NUMBER'] != st.session_state.data.at[index, 'PHONE_NUMBER']:
                update_data_in_snowflake(st.session_state.data.at[index, 'FULL_NAME'], row['FULL_NAME'], row['PHONE_NUMBER'])
                st.success(f"Updated: {row['FULL_NAME']}, Phone Number = {row['PHONE_NUMBER']}")
            else:
                st.info(f"No changes for: {row['FULL_NAME']}")

        # Update the session state data with the edited data after saving changes
        st.session_state.data = edited_data  # Store the updated data in session state

        # Refresh the page to reflect the changes immediately
        st.rerun()  # This will rerun the app and reload the table with updated data

    except Exception as e:
        st.error(f"Error updating data: {str(e)}")